In [1]:
import pandas as pd
import jieba
import re
import numpy as np

c:\users\user\anaconda3\envs\fda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\anaconda3\envs\fda\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\user\anaconda3\envs\fda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv("Train_risk_classification_ans.csv")
df = df.drop(columns= "Unnamed: 0")

In [3]:
# 斷詞後刪掉 "的"
counts = {}
jieba.load_userdict('./userdict.txt')
for i in range(df.shape[0]):
    df["text"][i] = jieba.lcut(df["text"][i])
    for j in range(len(df["text"][i])):
        if df["text"][i][j] == "的":
            df["text"][i][j] = ""
    df["text"][i] = ''.join(df["text"][i])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 0.766 seconds.
Prefix dict has been built successfully.
c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [4]:
# 刪除贅字
def replace_name(row):
    row.text = re.sub(":五。",":嗯。", row.text) # 台語
    row.text = re.sub(":a[,。]",":欸。", row.text)
    row.text = re.sub("[,。]a[,。]",":欸。", row.text) 
    row.text = re.sub("[(醫師)|人]?[甲乙]?醫師[\w]?:","", row.text) 
    row.text = re.sub("家屬[\w]?:","", row.text) 
    row.text = re.sub("民眾[\w]?:","", row.text)
    row.text = re.sub("護理師[\w]?:","", row.text)
    row.text = re.sub('個管師[\w]?:', '', row.text)  
    row.text = re.sub("[ab]先生","先生", row.text)
    row.text = re.sub('[阿啊痾啦呀啦嘛哇吧ㄟ欸诶誒耶餒咧嘞勒了嗎呢唔]', '', row.text)
    row.text = re.sub('[哼亨蛤蝦呃呵哈嘿喔哦唷呦喲齁啦摁恩嗯哎唉]', '', row.text)    
    row.text = row.text.replace('挂', '掛')
    row.text = row.text.replace('如果假設', '如果')
    row.text = row.text.replace('不應該', '不該')
    row.text = row.text.replace('了解', '瞭解')
    row.text = row.text.replace('甚麼', '什麼')
    row.text = row.text.replace('艾滋病', '愛滋病')
    row.text = row.text.replace('固砲', '固炮')
    row.text = row.text.replace('就是說', '')
    row.text = row.text.replace('24', '二十四')
    row.text = row.text.replace('\u200b', '')
    row.text = re.sub('(然後)|(所以)|(而且)|(其實)|(大概)|(應該)|(反正)|(所謂)|(一下)', '', row.text)    
    row.text = re.sub("就是?", "", row.text)
    row.text = re.sub("這樣子?", "", row.text)  
    row.text = re.sub('(ok)+', '好', row.text)
    row.text = re.sub('好+', '好', row.text)
    row.text = re.sub('我+', '我', row.text)
    row.text = re.sub('對+', '對', row.text)    
    row.text = re.sub('會+', '會', row.text)
    row.text = re.sub('謝+', '謝', row.text)
    row.text = re.sub('又+', '又', row.text)
    row.text = re.sub('都+', '都', row.text)
    row.text = re.sub('不+', '不', row.text)
    row.text = re.sub('是+', '是', row.text)
    
    row.text = re.sub("[0-9]+", '', row.text)
    row.text = re.sub("[。、…⋯“”]", '', row.text)
    row.text = re.sub("[,!:~·‧]", '', row.text)
    row.text = re.sub("[\?\.]", '', row.text)
    return row

In [5]:
def strQ2B(row):
    # 轉小寫
    row.text = row.text.lower()
    # 全形轉半形
    rstring = ""
    for uchar in row.text:
        u_code = ord(uchar)
        if (65281 <= u_code <= 65374):
            u_code -= 65248
        rstring += chr(u_code)
    row.text = rstring
    return row

In [6]:
df1 = df.apply(strQ2B, axis=1)
df1 = df1.apply(replace_name, axis=1)
df1 = df1.apply(replace_name, axis=1)

In [ ]:
!pip install opencc

In [7]:
#轉簡體
from opencc import OpenCC
cc = OpenCC('tw2sp')
for i in range( len( df1.index ) ):
    df1['text'][i] = cc.convert( df1['text'][i] )
df1.head()

c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,article_id,text,label
0,1,这个月还好这个月还好这个月还好还好还可以有性行为有是跟固定伴侣对固定伴侣对你有固定伴侣啰本来...,1
1,2,你这个月还好还好那有固定伴侣没有你一直以来都没有固定伴侣什么你从一开始没有固定伴侣你说来这里...,1
2,3,在忙装潢干对都在熬夜对都一直熬夜也没有熬夜回家我吃安眠药睡对为什么看起来没有精神是安眠药还没...,1
3,4,对你是说你要帮我帮你给这个你说小花跟你现在是状况是还好比较差是我在那个line上面那个时候那...,1
4,5,好过去这一个月还好还可以那今天是请假没有晚上上班晚上上班上什么班调酒师好那现在还是固定伴侣对...,0


In [8]:
# delete lenth > 3000
# 因為在刪除贅字後還超過3000字的文章代表內容太冗缺乏重點，為避免給模型造成過多雜訊，將之移除
drop_long = []
for i in range(df1.shape[0]):
    if len(df1["text"][i]) > 3000:
        drop_long.append(i)
len(drop_long)

11

In [9]:
df2 = df1.drop(index = drop_long)
df2 = df2.reset_index(drop = True)

In [10]:
label = []
article_id = []
text = []
bin_size = 460
buffer_size = 50

In [11]:
# 將文章切成每段510字，前後50字會和相鄰資料重疊
for i in range( len( df2.index )):
    buffer = ""
    bin_count = len( df2["text"][i] ) // bin_size + 1
    for j in range( bin_count - 1 ):
        label.append( df2["label"][i] )
        article_id.append( df2["article_id"][i] )

        text.append( buffer + df2["text"][i][ bin_size * j : bin_size * (j+1) ]  )

        buffer = df2["text"][i][ bin_size * (j+1) - 50 : bin_size * (j+1) ]

    label.append( df2["label"][i] )
    article_id.append( df2["article_id"][i] )
    text.append( df2["text"][i][ bin_size * (bin_count - 1):] )

In [12]:
df3 = pd.DataFrame(columns = ['article_id', 'label', 'text'] )
df3['article_id'] = article_id
df3['label'] = label
df3['text'] = text

In [13]:
# delete length < 50
# 小於50字的句子代表內容在前一筆資料已經有(因為前後會overlap 50字)，因此刪除
drop_list = []
for i in range(df3.shape[0]):
    if len(df3["text"][i]) < 50:
        drop_list.append(i)
len(drop_list)

36

In [14]:
df4 = df3.drop(index = drop_list)
df4 = df4.reset_index(drop = True)
df4

,article_id,label,text
0,1,1,这个月还好这个月还好这个月还好还好还可以有性行为有是跟固定伴侣对固定伴侣对你有固定伴侣啰本来...
1,1,1,你会吃prep你会戴套对戴套对戴套全程对如果说全程的话也是没有那如果没有戴套是他要求还是你要...
2,1,1,没有很相信你固炮但我觉得是好因为保护自己因为prep本来不是吃给对方看不是一个证明文档说我有...
3,2,1,你这个月还好还好那有固定伴侣没有你一直以来都没有固定伴侣什么你从一开始没有固定伴侣你说来这里...
4,2,1,还是比较少一点偶尔戴套意思是那吃prep之后更少比偶尔还要再少一点戴套对你会觉得吃prep是...
...,...,...,...
868,344,0,好你这次抽血都还不错看是年度检查我们下次一年后再抽血可以好那下次回来打流感下次回诊时候你马上...
869,345,0,我还要问是问医师一个问题我肝药你给我开两个月看两个月药钱肝药对肝药你帮我看算起来差不多多少钱...
870,345,0,冒比较咳嗽有咳嗽比较会咳嗽比较会咳嗽是这三个礼拜比较会咳嗽我一咳嗽出来的话要喘气很困难我觉得...
871,346,0,现在是在哪在也是在念硕班对要毕业希望是快希望快你要是电机系对是做什么研究我是做微电路组织微电...


In [15]:
df4.to_csv("risk_df_final_sp.csv", index = False)

In [16]:
# test
df = pd.read_csv("Test_risk_classification.csv")
df.head()

,Unnamed: 0,article_id,text,label
0,1,1,"醫師:差不多這樣齁。我看眼睛甚麼時候能夠檢查。年底啦齁。民眾:是的。醫師:許國強,病例是55...",NaN
1,2,2,"醫師:讓我們先檢查索引。啊該死,我可以檢查一下嗎?民眾:好的。醫師:好齁。啊電腦斷層以前沒有...",NaN
2,3,3,醫師:你去年有打。民眾:我去年有打。民眾:嗯。醫師:這種免疫力。民眾:好啊。醫師:太可怕了。...,NaN
3,4,4,"醫師:像這樣。民眾:嘿,照這樣呵。啊但是,我現在就是,也不一定是照這樣我就不會發作阿。醫師:...",NaN
4,5,5,醫師:好像有點低。民眾:按。醫師:所以好像沒有過敏。民眾:不。醫師:所以可能那些心悸是吃那個...,NaN


In [17]:
# 斷詞後刪掉 "的"
counts = {}
jieba.load_userdict('./userdict.txt')
for i in range(df.shape[0]):
    df["text"][i] = jieba.lcut(df["text"][i])
    for j in range(len(df["text"][i])):
        if df["text"][i][j] == "的":
            df["text"][i][j] = ""
    df["text"][i] = ''.join(df["text"][i])

c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
df1 = df.apply(strQ2B, axis=1)
df2 = df1.apply(replace_name, axis=1)
df2 = df2.apply(replace_name, axis=1)

In [21]:
for i in range( len( df2.index ) ):
    df2['text'][i] = cc.convert( df2['text'][i] )
df2.head()

c:\users\user\anaconda3\envs\fda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,article_id,text,label
0,1,1,差不多我看眼睛什么时候能够检查年底是许国强病例是好下礼拜那我们也是照样照照常纪录好那个伤口让...,NaN
1,2,2,让我们先检查索引该死我可以检查好电脑断层以前没有拍过没有计算机断层扫描我没有拍出好照片我很热...,NaN
2,3,3,你去年有打我去年有打这种免疫力好太可怕你认为我会在月日之前哪一天他周三有约会他是周三周五会更...,NaN
3,4,4,像照但是我现在也不一定是照我不会发作是对那我到底错过什么不这么红小华等我们测耳朵和压舌板温度...,NaN
4,5,5,好像有点低按好像没有过敏不可能那些心悸是吃那个药会不会是现在也不好分细菌在是我觉得这很糟糕我...,NaN


In [22]:
article_id = []
text = []
for i in range( len( df2.index )):
    buffer = ""
    bin_count = len( df2["text"][i] ) // bin_size + 1
    for j in range( bin_count - 1 ):
        label.append( df2["label"][i] )
        article_id.append( df2["article_id"][i] )

        text.append( buffer + df2["text"][i][ bin_size * j : bin_size * (j+1) ]  )

        buffer = df2["text"][i][ bin_size * (j+1) - 50 : bin_size * (j+1) ]

    article_id.append( df2["article_id"][i] )
    text.append( df2["text"][i][ bin_size * (bin_count - 1):] )

In [23]:
df3 = pd.DataFrame(columns = ['article_id', 'label', 'text'] )
df3['article_id'] = article_id
df3['text'] = text

In [24]:
drop_list = []
for i in range(df3.shape[0]):
    if len(df3["text"][i]) < 50:
        drop_list.append(i)
len(drop_list)

397

In [25]:
df4 = df3.drop(index = drop_list)
df4.to_csv("risk_df_final_test_sp.csv")